In [1]:
"""
    Author: Nicolas Bulté
"""

"""
Glossary mapping
"""

# LoanID               = Loan Identifier (A,P)
# MonthRep             = Monthly Reporting Period (P)
# Servicer             = Servicer Name (P)
# CurrInterestRate     = CURRENT INTEREST RATE (P)
# CAUPB                = CURRENT ACTUAL UNPAID PRINCIPAL BALANCE (P)
# LoanAge              = Loan Age (P)
# MonthsToMaturity     = Remaining Months to Legal Maturity (P)
# AdMonthsToMaturity   = ADJUSTED REMAINING MONTHS TO MATURITY (P)
# MaturityDate         = Maturity Date (P)
# MSA                  = Metropolitan Statistical Area (P)
# CLDS                 = Current Loan Delinquency Status (P)
# ModFlag              = Modification Flag (P)
# ZeroBalCode          = Zero Balance Code (P)
# ZeroBalDate          = Zero Balance Effective Date(P)
# LastInstallDate      = LAST PAID INSTALLMENT DATE
# ForeclosureDate      = FORECLOSURE DATE
# DispositionDate      = DISPOSITION DATE
# ForeclosureCosts     = FORECLOSURE COSTS (P)
# PPRC                 = Property Preservation and Repair Costs (P)
# AssetRecCost         = ASSET RECOVERY COSTS (P)
# MHEC                 = Miscellaneous Holding Expenses and Credits (P)
# ATFHP                = Associated Taxes for Holding Property (P)
# NetSaleProceeds      = Net Sale Proceeds (P)
# CreditEnhProceeds    = Credit Enhancement Proceeds (P)
# RPMWP                = Repurchase Make Whole Proceeds(P)
# OFP                  = Other Foreclosure Proceeds (P)
# NIBUPB               = Non-Interest Bearing UPB (P)
# PFUPB                = PRINCIPAL FORGIVENESS UPB (P)
# RMWPF                = Repurchase Make Whole Proceeds Flag (P)
# FPWA                 = Foreclosure Principal Write-off Amount (P)
# ServicingIndicator   = SERVICING ACTIVITY INDICATOR (P)

"""
    Import statements 
"""

import pandas as pd
import numpy as np
import datetime as dt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, roc_curve, f1_score, average_precision_score


# Import datasets, select features and define the default-flag column.
col_per = ['LoanID', 'MonthRep', 'Servicer', 'CurrInterestRate', 'CAUPB', 'LoanAge', 'MonthsToMaturity',
           'AdMonthsToMaturity', 'MaturityDate', 'MSA', 'CLDS', 'ModFlag', 'ZeroBalCode', 'ZeroBalDate',
           'LastInstallDate', 'ForeclosureDate', 'DispositionDate', 'ForeclosureCosts', 'PPRC', 'AssetRecCost', 'MHEC',
           'ATFHP', 'NetSaleProceeds', 'CreditEnhProceeds', 'RPMWP', 'OFP', 'NIBUPB', 'PFUPB', 'RMWPF',
           'FPWA', 'ServicingIndicator']

# Python will guess the datatypes not specified in the map function, for dates the dtype will be 'object'. (hence: here all dates)
# If an expected integer variables contains NaN values it will be set to 'float32'
perf_type_map = {'LoanID': 'int64', 'Servicer': 'category', 'CurrInterestRate': 'float32', 'CAUPB': 'float32',
                 'LoanAge': 'int64', 'MonthsToMaturity': 'int64', 'AdMonthsToMaturity': 'float32', 'MSA': 'category',
                 'CLDS': 'category', 'ModFlag': 'category', 'ZeroBalCode': 'float32', 'ForeclosureCosts': 'float32',
                 'PPRC': 'float32', 'AssetRecCost': 'float32', 'MHEC': 'float32', 'ATFHP': 'float32',
                 'NetSaleProceeds': 'float32', 'CreditEnhProceeds': 'float32', 'RPMWP': 'float32', 'OFP': 'float32',
                 'NIBUPB': 'float32', 'PFUPB': 'float32', 'RMWPF': 'category', 'FPWA': 'float32',
                 'ServicingIndicator': 'category'}

extended_selec_per = col_per

col_per_subset = extended_selec_per

In [2]:
file_name='C:/Users/bebxadvberb/Documents/AI/Trusted AI/Performance_HARP.txt'
lines_to_read = 1e5

In [3]:
pf = pd.read_csv(file_name, sep='|', names=col_per, index_col=False,
                     nrows=lines_to_read)
pf['CLDS'] = pf.CLDS.replace('X', '1').astype('float')

C:\Users\bebxadvberb\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def create_12mDefault(date, perf_df):
    """
    Create the 12 month forward looking default flag.
    Parameters
    ----------
    date: Snapshot date
    perf_df: Performance dataframe
    Returns
    -------
    Raw observation dataframe
    """
    cur_date = dt.datetime.strptime(date, '%m/%d/%Y').date()
    # Fix the IDs in the observation set by fixing their reporting date AND requiring that the files are healthy.
    obs_df = perf_df[(perf_df.MonthRep == date)
                     &
                     ((perf_df.CLDS == 0.0) |
                      (perf_df.CLDS == 1.0) |
                      (perf_df.CLDS == 2.0)
                      )
                     ]
    obs_ids = obs_df.LoanID
    # Load only the observation IDs in the performance frame initially.
    pf = perf_df[perf_df.LoanID.isin(obs_ids)]

    # Create the 12 month forward looking list of dates
    date_list = []
    for i in np.arange(0, 12):
        if cur_date.month == 12:
            month = 1
            year = cur_date.year + 1
        else:
            month = cur_date.month + 1
            year = cur_date.year
        next_date = dt.datetime(year, month, cur_date.day)
        date_list.append(next_date.strftime('%m/%d/%Y'))
        cur_date = next_date

    # Find the LoanIDs of those loans where a default appears in our 12 month forward looking period.
    pf_obs = perf_df[perf_df.MonthRep.isin(date_list)]
    pf_obs_defaults = pf_obs[
        (pf_obs.CLDS != 0.0) &
        (pf_obs.CLDS != 1.0) &
        (pf_obs.CLDS != 2.0)
        ].LoanID

    pf_obs_defaults = pf_obs_defaults.drop_duplicates(keep='last').values
    df = obs_df
    df['Default'] = 0
    df.loc[df['LoanID'].isin(pf_obs_defaults), 'Default'] = 1

    return df

In [5]:
def select_sample(observ_df):
    '''
    Select randomly 1/8 of the accounts from each of the 8 quarterly snapshot; an account should appear only once. 
    This way, the final sample will have an even mix of each quarter and will be equivalent size of the portfolio 
    on average over the 2 years.
    Parameters
    ----------
    observ_df: observation dataframe
    Returns
    -------
    '''
    snapshots = observ_df.MonthRep.unique()
    # Store the size of (in case of 8 snapshot dates) 1/8 of the dataset, divided by 8 again to get the size of 1/8 of a snapshot set. 
    # Use this to sample from every snapshot set to come to a final df that contains the 1/8 of the original size and has equal
    # contribution of every quarter/snapshot moment. 
    i = int(observ_df.shape[0] / len(snapshots) / len(snapshots))
    l = []
    for d in snapshots:
        l.append(observ_df[observ_df.MonthRep == d].sample(n=i, replace=False, random_state=1))
    df = pd.concat(l)
    return df

In [6]:
def traintest_split(observation_frame, testsize=0.2):
    X = observation_frame.drop('Default', axis=1)
    Y = observation_frame.Default
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=testsize, random_state=1)
    return X_train, X_test, y_train, y_test

In [7]:
# Define your snapshot dates for your observation frame:
date_list = ['03/01/2016', '06/01/2016', '09/01/2016', '12/01/2016', '03/01/2017', '06/01/2017', '09/01/2017',
             '12/01/2017']
observation_frame = pd.concat([create_12mDefault(d, pf) for d in date_list])

C:\Users\bebxadvberb\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\bebxadvberb\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [8]:
observation_frame = observation_frame[observation_frame.CAUPB.notnull()]
observation_frame = observation_frame[observation_frame.AdMonthsToMaturity.notnull()]

In [9]:
X_train, X_test, y_train, y_test = traintest_split(observation_frame)

In [10]:
X,y = X_train, y_train

In [11]:
X.head()

,LoanID,MonthRep,Servicer,CurrInterestRate,CAUPB,LoanAge,MonthsToMaturity,AdMonthsToMaturity,MaturityDate,MSA,...,ATFHP,NetSaleProceeds,CreditEnhProceeds,RPMWP,OFP,NIBUPB,PFUPB,RMWPF,FPWA,ServicingIndicator
1015,100021670447,09/01/2016,NaN,4.500,75835.89,66,114,107.0,03/2026,15940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
40278,100609128433,09/01/2017,NaN,4.375,114387.35,43,137,130.0,02/2029,16980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
51115,100760526472,06/01/2017,NaN,4.500,156629.62,55,305,305.0,11/2042,41860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
20358,100323063324,12/01/2016,NaN,4.625,136235.85,55,305,304.0,05/2042,35620,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
71939,101056738742,06/01/2017,NaN,3.500,108190.82,52,188,187.0,02/2033,12060,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N


# Start From this cell

In [12]:
y_train.value_counts(1)

0    0.989713
1    0.010287
Name: Default, dtype: float64

In [13]:
def get_na_feat(df):
    na_columns = df.columns[df.isnull().any()]
    return na_columns

In [14]:
# DROP FEATURES WITH NA VALUES
na_columns = get_na_feat(X)
X = X.drop(na_columns,axis=1)

In [15]:
def get_cat_feat(df):
    cat_feat = df.select_dtypes(include=['object']).columns
    return cat_feat

def get_num_feat(df):
    num_feat = df.select_dtypes(exclude=['object']).columns
    return num_feat

In [16]:
def label_encode(df):
    df = df.apply(LabelEncoder().fit_transform)
    return df

def one_hot_encode(df):
    enc = OneHotEncoder(handle_unknown='ignore')
    df = enc.transform(df)
    return df

In [17]:
X = X.drop('ModFlag', axis=1)

In [18]:
# FIND THE CATEGORICAL FEATURES
cat_feat = get_cat_feat(X)

for cat in cat_feat:
    X[cat] = LabelEncoder().fit_transform(X[cat])
    
X.head()

,LoanID,MonthRep,CurrInterestRate,CAUPB,LoanAge,MonthsToMaturity,AdMonthsToMaturity,MaturityDate,MSA,CLDS
1015,100021670447,4,4.500,75835.89,66,114,107.0,35,15940,0.0
40278,100609128433,5,4.375,114387.35,43,137,130.0,20,16980,0.0
51115,100760526472,3,4.500,156629.62,55,305,305.0,199,41860,0.0
20358,100323063324,6,4.625,136235.85,55,305,304.0,83,35620,0.0
71939,101056738742,3,3.500,108190.82,52,188,187.0,24,12060,0.0


In [19]:
X.head()

,LoanID,MonthRep,CurrInterestRate,CAUPB,LoanAge,MonthsToMaturity,AdMonthsToMaturity,MaturityDate,MSA,CLDS
1015,100021670447,4,4.500,75835.89,66,114,107.0,35,15940,0.0
40278,100609128433,5,4.375,114387.35,43,137,130.0,20,16980,0.0
51115,100760526472,3,4.500,156629.62,55,305,305.0,199,41860,0.0
20358,100323063324,6,4.625,136235.85,55,305,304.0,83,35620,0.0
71939,101056738742,3,3.500,108190.82,52,188,187.0,24,12060,0.0


In [20]:
from imblearn.under_sampling import RandomUnderSampler
sm = RandomUnderSampler()

X_cols = X.columns
X, y = sm.fit_sample(X, y) # fit_sample takes a dataframe, but returns an array. 
(X, y) = (pd.DataFrame(X, columns=X_cols), pd.Series(y))
print(y.value_counts(1))

1    0.5
0    0.5
dtype: float64


C:\Users\bebxadvberb\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [21]:
X.count()

LoanID                140
MonthRep              140
CurrInterestRate      140
CAUPB                 140
LoanAge               140
MonthsToMaturity      140
AdMonthsToMaturity    140
MaturityDate          140
MSA                   140
CLDS                  140
dtype: int64

In [22]:
def normalize(df):
    df_norm = df
    df_norm = df_norm.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
    return df_norm

In [23]:
y.count()

140

In [24]:
# DIVIDE THE DATA IN 10 STRATIFIED FOLDS
skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(X, y)

10

In [25]:
from interpret.glassbox import ExplainableBoostingClassifier

ebm = ExplainableBoostingClassifier()
ebm.fit(X_train, y_train)

ModuleNotFoundError: No module named 'interpret'

In [80]:
for train_index, test_index in skf.split(X, y):
    
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    print(y_train.isnull().values.any())
    
    # fit model on training data
    model.fit(X_train, y_train)
    
    # make predictions for test data
    y_pred = model.predict(X_test)
    # temp = pd.concat([X_test,y_pred], axis=1)
    print(y_pred)
    predictions = [round(value) for value in y_pred]
    
    # evaluate predictions
    auc = roc_auc_score(y_test, predictions)
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions)
    print("Accuracy: %.2f%%" % (accuracy * 100.0), "|| AUC: %.2f%%" % (auc * 100.0), "|| F1 - Score: %.2f%%" % (f1 * 100.0))

False


C:\Users\bebxadvberb\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[-0.02798283  0.4521431   0.67808044  0.26532     0.56221527  0.31967956
 -0.06507522  1.038782    0.8808931   0.94542     0.9925295   0.9155842
  0.82671833  1.0207849 ]
Accuracy: 85.71% || AUC: 85.71% || F1 - Score: 87.50%
False


C:\Users\bebxadvberb\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[ 0.4591816   0.274427    0.5595084   0.28330478 -0.03493243  0.5787569
 -0.06953382  0.7872021   0.8583796   1.1342161   0.74283123  0.9151305
  0.8267276   0.74053526]
Accuracy: 85.71% || AUC: 85.71% || F1 - Score: 87.50%
False


...

[ 0.3146611   0.12271816  0.92907226  0.45134017  0.8461002   0.6482932
 -0.19865572  0.8465036   0.79452944  0.91433     0.89756995  0.7338531
  0.45958775  0.47951135]
Accuracy: 64.29% || AUC: 64.29% || F1 - Score: 66.67%
False


C:\Users\bebxadvberb\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[0.35228533 0.0839234  0.40894008 0.13265938 0.01467201 0.74372685
 0.5059902  0.9783639  0.11464751 1.0543982  0.6381585  1.0189545
 0.7725555  0.8498508 ]
Accuracy: 78.57% || AUC: 78.57% || F1 - Score: 80.00%
False


C:\Users\bebxadvberb\AppData\Local\Continuum\anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[ 0.27126735  0.47365355  0.7239984   0.01690772  0.17814553  0.13718495
 -0.04920715  0.7715382   1.1739364   0.96702003  1.0790238   0.78940666
  0.63942784  0.84061635]
Accuracy: 92.86% || AUC: 92.86% || F1 - Score: 93.33%


In [81]:
len(train_index)

126

In [26]:
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

boston = load_boston()
feature_names = list(boston.feature_names)
X, y = pd.DataFrame(boston.data, columns=feature_names), boston.target

seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

In [ ]:
from interpret import show
from interpret.data import Marginal

marginal = Marginal().explain_data(X_train, y_train, name = 'Train Data')
show(marginal)

In [27]:
from interpret.glassbox import ExplainableBoostingRegressor, LinearRegression, RegressionTree

ebm = ExplainableBoostingRegressor(random_state=seed)
ebm.fit(X_train, y_train)   #Works on dataframes and numpy arrays

ModuleNotFoundError: No module named 'interpret'

In [28]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

NameError: name 'ebm' is not defined